In [48]:
%load_ext autoreload
%autoreload 2
import data_loader

from feature_extraction import extract_features, data_in_tw
#from datetime import datetime, timedelta
import pandas as pd
import pytz
import pickle

from sklearn import svm
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Size of time window in unit of second.
TIME_WINDOW_SEC = 30
features = []
labels = []

# Data Loading

In [3]:
dl = data_loader.DataLoader()

# Load Groudtruth
df_labels = dl.load_groundtruth()


# Load low frequency data
df_location = dl.load_low_freq_data('LOCATION')
df_activity = dl.load_low_freq_data('ACTIVITY_TYPE')

Loading /Users/zxxia/Research/bharath-data/lowfreqdata/10_null_LOCATION_null_PHONE_null_null_org.md2k.phonesensor_null_DATA.csv...
Loading /Users/zxxia/Research/bharath-data/lowfreqdata/11_null_ACTIVITY_TYPE_null_PHONE_null_null_org.md2k.phonesensor_null_DATA.csv...


In [4]:
visited = set()
for index, row in df_labels.iterrows():
    d = row['Start Timestamp'].date()
    if d not in visited:
        # load high frequency data
        df_battery = dl.load_high_freq_data('BATTERY', d)
        df_compass = dl.load_high_freq_data('COMPASS', d)
        visited.add(d)

    start = row['Start Timestamp'] # + timedelta(seconds=120)
    end = row['End Timestamp'] # - timedelta(seconds=120)

    if start >= end:
        continue

    # Obtain data needed for one label(one event)
    streams = {}
    streams['Battery'] = data_in_tw(df_battery, start, end)
    streams['Compass'] = data_in_tw(df_compass, start, end)
    streams['Activity'] = data_in_tw(df_activity, start, end)
    streams['Location'] = data_in_tw(df_location, start, end)

    # Segment data streams into time windows, extract features and append features
    tmp = extract_features(streams, start, end, TIME_WINDOW_SEC)
    features = features + tmp
    
    # Extract labels and append labels
    labels = labels + [row['Labels']] * len(tmp)

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012000_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012001_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012002_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012003_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012004_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012005_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012006_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012007_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012008_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012009_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012010_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012011_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012012_9.csv.gz...
Loading /Users/zxxia/Rese

Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012214_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012215_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012216_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012217_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012218_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012219_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012220_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012221_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012222_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012223_14.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012300_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012301_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012302_9.csv.gz...
Loadi

Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012508_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012509_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012510_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012511_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012512_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012513_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012514_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012515_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012516_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012517_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012518_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw9/2018012519_9.csv.gz...
Loading /Users/zxxia/Research/bharath-data/raw/raw14/2018012500_14.csv.gz...
Loading /Users/zxxia/Re

In [5]:
print(len(features))
print(len(labels))

15107
15107


# Data Preprocessing

In [54]:
# Convert features and labels to pandas dataframe format
df_features = pd.DataFrame(features)
df_features['Labels'] = pd.DataFrame(labels)

# df_features.to_csv('zhengxux_features_labels.csv')

df_features['Battery Charging'].interpolate(method='nearest', inplace=True)
df_features['Max GPS Accuracy'].fillna(500, inplace=True)
df_features = df_features.dropna(axis=0)


# Separate entire dataframe into different classes
df_incar = df_features[df_features.Labels=='Incar']
df_indoor = df_features[df_features.Labels=='Indoor']
df_outdoor = df_features[df_features.Labels=='Outdoor']

# Downsample majority class
df_indoor = resample(df_indoor, 
                     replace=False,   # sample without replacement
                     n_samples=613,   # to match minority class
                     random_state=42) # reproducible results

df_features = pd.concat([df_indoor, df_outdoor, df_incar])

df_labels = df_features['Labels']
df_features.drop('Labels', axis=1, inplace=True)




# train_features is numpy array
vec = DictVectorizer()
train_features = vec.fit_transform(df_features.to_dict('records')).toarray()

# Normalize columns of features with min-max scaler
scaler = MinMaxScaler(copy=False)
train_features = scaler.fit_transform(train_features)

# Encode all features
le = LabelEncoder()
le.fit(['Incar', 'Indoor', 'Outdoor'])
train_labels = le.transform(df_labels)

In [56]:
print(df_indoor.shape)
print(df_outdoor.shape)
print(df_incar.shape)
print(df_labels.shape)
print(df_features.shape)

(613, 18)
(613, 18)
(584, 18)
(1810,)
(1810, 17)


# Training on SVM

In [65]:
# 10 fold corss validation
clf = svm.SVC()
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    confusionmatrix = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:\n', confusionmatrix)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Micro F1 Score:', f1_score(y_test, y_pred, average='micro'))
    print('Macro F1 Score:', f1_score(y_test, y_pred, average='macro'))
    print('Weighted F1 Score:', f1_score(y_test, y_pred, average='weighted'))
    print('\n')


Confusion Matrix:
 [[34 11 10]
 [ 5 54 12]
 [ 2  5 48]]
Accuracy: 0.75138121547
Micro F1 Score: 0.75138121547
Macro F1 Score: 0.747430260047
Weighted F1 Score: 0.749068022413


Confusion Matrix:
 [[54  6  5]
 [ 3 54  5]
 [ 2 11 41]]
Accuracy: 0.82320441989
Micro F1 Score: 0.82320441989
Macro F1 Score: 0.821316732692
Weighted F1 Score: 0.823923737342


Confusion Matrix:
 [[49  8  4]
 [ 6 46  2]
 [ 4 19 43]]
Accuracy: 0.762430939227
Micro F1 Score: 0.762430939227
Macro F1 Score: 0.762967400814
Weighted F1 Score: 0.764040323989


Confusion Matrix:
 [[46 10  2]
 [ 7 48 14]
 [ 1  9 44]]
Accuracy: 0.762430939227
Micro F1 Score: 0.762430939227
Macro F1 Score: 0.766413582977
Weighted F1 Score: 0.762612983548


Confusion Matrix:
 [[41 20  3]
 [ 2 61  8]
 [ 1 13 32]]
Accuracy: 0.740331491713
Micro F1 Score: 0.740331491713
Macro F1 Score: 0.73925144075
Weighted F1 Score: 0.74126085069


Confusion Matrix:
 [[40  4  4]
 [ 4 62  3]
 [ 2 21 41]]
Accuracy: 0.790055248619
Micro F1 Score: 0.790055248619

In [9]:
# save the model to disk
filename = 'bhararh_model.clf'
pickle.dump(clf, open(filename, 'wb'))

In [15]:
# load the model from disk
filename = '../clf/zhengxuxia_model.clf'
zhengxux_model = pickle.load(open(filename, 'rb'))
y_pred = zhengxux_model.predict(train_features)
confusionmatrix = confusion_matrix(train_labels, y_pred)
print(confusionmatrix)
print(accuracy_score(train_labels, y_pred, normalize=True))
print(f1_score(train_labels, y_pred, average='micro'))
print(f1_score(train_labels, y_pred, average='macro'))
print(f1_score(train_labels, y_pred, average='weighted'))

[[ 372  184   28]
 [  44 9748  421]
 [   2  283  328]]
0.915687992989
0.915687992989
0.722944587432
0.917613942646


In [25]:
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
svd.fit_transform(train_features)

ValueError: n_components must be < n_features; got 17 >= 17

In [24]:
print(svd.explained_variance_ratio_)
print(train_features.shape)

[  8.19503902e-02   2.68519859e-01   2.11723393e-01   1.14381935e-01
   1.00368488e-01   7.42230008e-02   6.51592453e-02   3.59981399e-02
   1.71882410e-02   1.24708370e-02   7.16832240e-03   4.71678926e-03
   3.25469560e-03   2.87666404e-03   7.33620520e-36   2.78095947e-64]
(11410, 17)
